In [1]:
%run requirements.ipynb
%run config.ipynb
%run module_training.ipynb

In [ ]:
"""
start_time = timer(None)
timer(start_time)
"""

In [2]:
def v_value(smart_meter_data_group, NUMBER_OF_DAYS):
    CONDITION_FILTER = 28
    v_value_list = []
    v_value_df = pd.DataFrame(columns=['MeterID', 'Day', 'FDI', 'Tmax1', 'Cmax1', 'Tmax2', 'Cmax2', 'Tmin1', 'Cmin1'])
    pbar = tqdm(smart_meter_data_group) 
    for area in pbar: # iterate 10 data groups
        for meter in area['fdi']: # iterate every fdi meter in area 
            day_list = meter['Day'].reset_index()
            for i in range(NUMBER_OF_DAYS): # iterate every day
                meter_id = meter['MeterID'].max()
                meter_day = day_list['Day'][i*48]
                meter_numpy = meter['Electricity'].to_numpy()[i*48:(i+1)*48]

                # 1. Cmax1 and T max1 are the maximum consumption before 2 p.m.
                Tmax1 = meter_numpy[:CONDITION_FILTER].argmax()

                # 2. Cmax2 and T max2 are the maximum consumption after 2 p.m.
                Tmax2 = meter_numpy[CONDITION_FILTER:].argmax() + CONDITION_FILTER

                # 3. Cmin1 and T min1 are the minimum consumption between T max1 and T max2 
                Tmin1 = meter_numpy[Tmax1: Tmax2].argmin() + Tmax1

                se = {'MeterID': meter_id, 'Day': meter_day,'FDI': True, 'Tmax1': Tmax1, 'Tmax2': Tmax2, 'Tmin1': Tmin1, 'Cmax1': meter_numpy[Tmax1], 'Cmax2': meter_numpy[Tmax2], 'Cmin1': meter_numpy[Tmin1]}
                v_value_list.append(se)
        print('finish fdi')
        for meter in area['normal']:
            day_list = meter['Day'].reset_index()
            for i in range(NUMBER_OF_DAYS): # iterate every day
                meter_id = meter['MeterID'].max()
                meter_day = day_list['Day'][i*48]
                meter_numpy = meter['Electricity'].to_numpy()[i*48:(i+1)*48]
    #             print(meter_numpy)
                # 1. Cmax1 and T max1 are the maximum consumption before 2 p.m.
                Tmax1 = meter_numpy[:CONDITION_FILTER].argmax()

                # 2. Cmax2 and T max2 are the maximum consumption after 2 p.m.
                Tmax2 = meter_numpy[CONDITION_FILTER:].argmax() + CONDITION_FILTER

                # 3. Cmin1 and T min1 are the minimum consumption between T max1 and T max2 
                Tmin1 = meter_numpy[Tmax1: Tmax2].argmin() + Tmax1

                se = {'MeterID': meter_id, 'Day': meter_day,'FDI': False, 'Tmax1': Tmax1, 'Tmax2': Tmax2, 'Tmin1': Tmin1, 'Cmax1': meter_numpy[Tmax1], 'Cmax2': meter_numpy[Tmax2], 'Cmin1': meter_numpy[Tmin1]}
                v_value_list.append(se)
        print('finish normal')
    return v_value_list

In [ ]:
# four_time_list = four_time(smart_meter_data_group)
def four_time(smart_meter_data_group, NUMBER_OF_DAYS):
    four_time_list = []
    fourtime_df = pd.DataFrame(columns=['MeterID', 'Day', 'FDI', 'overnight', 'breakfast', 'daytime', 'evening'])
    pbar = tqdm(smart_meter_data_group)
    count = 1
    for area in pbar: # iterate 10 data groups
        print(count)
        count += 1
        
        for meter in area['fdi']: # iterate every fdi meter in area 
            day_list = meter['Day'].reset_index()
            for i in range(NUMBER_OF_DAYS): # iterate every day
                meter_id = meter['MeterID'].max()
                meter_day = day_list['Day'][i*48]

                if i == 0:
                    meter_numpy = meter['Electricity'].to_numpy()[44:48]
                    meter_numpy = np.append(meter_numpy, meter['Electricity'].to_numpy()[i*48:(i+1)*48-4])
                else:
                    meter_numpy = meter['Electricity'].to_numpy()[i*48-4:(i+1)*48-4]

                overnight = np.mean(meter_numpy[0:17])
                breakfast = np.mean(meter_numpy[17:21])
                daytime = np.mean(meter_numpy[21:35])
                evening = np.mean(meter_numpy[35:48])
                se = {'MeterID': meter_id, 'Day': meter_day,'FDI': True, 'overnight': overnight, 'breakfast': breakfast, 'daytime': daytime, 'evening': evening}
                four_time_list.append(se)
        
        for meter in area['normal']: # iterate every fdi meter in area 
            for i in range(NUMBER_OF_DAYS): # iterate every day
                meter_id = meter['MeterID'].max()
                meter_day = day_list['Day'][i*48]

                if i == 0:
                    meter_numpy = meter['Electricity'].to_numpy()[44:48]
                    meter_numpy = np.append(meter_numpy, meter['Electricity'].to_numpy()[i*48:(i+1)*48-4])
                else:
                    meter_numpy = meter['Electricity'].to_numpy()[i*48-4:(i+1)*48-4]

                overnight = np.mean(meter_numpy[0:17])
                breakfast = np.mean(meter_numpy[17:21])
                daytime = np.mean(meter_numpy[21:35])
                evening = np.mean(meter_numpy[35:48])

                se = {'MeterID': meter_id, 'Day': meter_day,'FDI': False, 'overnight': overnight, 'breakfast': breakfast, 'daytime': daytime, 'evening': evening}
                four_time_list.append(se)
        return four_time_list